In [1]:
%pip install openai --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
  name="Code Documentor",
  instructions="You are an expert code documentor, you create code documents in markdown format, these documnent will be used to teach new commers and also to remember what is happening inside the code.  ",
  model="gpt-3.5-turbo",
  tools=[{"type": "file_search"}],
)

In [22]:
# Create a vector store caled "Financial Statements"
# vector_store = client.beta.vector_stores.create(name="Financial Statements")

# Ready the files for upload to OpenAI
# file_paths = ["push_to_confluence.py", "about_ptoc.txt"]
# file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#   vector_store_id=vector_store.id, files=file_streams
# )

# You can print the status and the file counts of the batch to see the result of this operation.
# print(file_batch.status)
# print(file_batch.file_counts)

files = client.beta.vector_stores.files.list(vector_store_id='vs_pezEMD2D2SjDwp18AHyRrF7Y')
for i, files in enumerate(files.data):
    print(f"{i+1} = {files}")
    # print(client.beta.vector_stores.files.retrieve(file_id=files.id, vector_store_id='vs_pezEMD2D2SjDwp18AHyRrF7Y'))

vector_store_id = ''
# assistant_id = ''

1 = VectorStoreFile(id='file-9CKi0nHlpgd5HGvypd65j05P', created_at=1715662037, last_error=None, object='vector_store.file', status='completed', usage_bytes=3176, vector_store_id='vs_pezEMD2D2SjDwp18AHyRrF7Y')
2 = VectorStoreFile(id='file-CXdDJXZwuohpusQtQuNj2qNE', created_at=1715662037, last_error=None, object='vector_store.file', status='completed', usage_bytes=1552, vector_store_id='vs_pezEMD2D2SjDwp18AHyRrF7Y')


In [26]:
# @title Default title text
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}},
)

In [27]:
# @title Default title text
# Upload the user provided file to OpenAI
# message_file = client.files.create(
#   file=open("file3.txt", "rb"), purpose="assistants"
# )
vector_store_id = ''
# assistant_id = ''

prompt = 'Generate a code document for the code push_to_confluence.py' # @param {type:"string"}
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": prompt ,
      # Attach the new file to the message.
      # "attachments": [
      #   { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      # ],
    }
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

None


In [28]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")



In [30]:
print(message_content.value)
print("\n".join(citations))
print(run.usage)

### Code Document for push_to_confluence.py

The `push_to_confluence.py` file is designed to push a markdown format file into a Confluence page. Since Confluence does not support direct pushing of markdown files, this script converts the markdown content into HTML before pushing it to Confluence. Here is an overview of the key functions and operations within the script:

#### Functions:
1. **clear_submit()**:
   - Function to set the submit state in the Streamlit session state.

2. **confluence()**:
   - Main function responsible for pushing content to Confluence.
   - Parameters:
     - `confluence_url`: URL of the Confluence instance.
     - `space_key`: Key of the Confluence space.
     - `username`: User name for authentication.
     - `api_token`: API token for authentication.
     - `page_title`: Title of the Confluence page.
     - `mark_data`: Content in markdown format to be converted and pushed to Confluence.
   - Converts the markdown content to HTML, prepares the data for t